In [ ]:
!pip3 install pandas
!pip3 install numpy
!pip3 install git+https://github.com/rcabanasdepaz/exPandas.git@main
!pip3 install logging-reset


In [1]:
import os
import pandas as pd
import numpy as np
from pathlib import Path
import logging
import logreset

import conf
#from experiments import *

In [2]:
conf.prj_path

'/root/dev/2021-SIPTA-crema/'

In [3]:
import subprocess
import datetime
import os

import pandas as pd



def gen_exec(cmd, check_return: bool = False):
    popen = subprocess.Popen(cmd, stdout=subprocess.PIPE, universal_newlines=True)
    for stdout_line in iter(popen.stdout.readline, ""):
        yield stdout_line
    popen.stdout.close()
    return_code = popen.wait()
    if return_code and check_return:
        raise subprocess.CalledProcessError(return_code, cmd)


def exec_bash(cmd: str, check_return: bool = False):
    return [s for s in gen_exec(cmd.split(), check_return)]
    
def exec_bash_print(cmd: str, check_return: bool = False):
    for path in gen_exec(cmd.split(), check_return):
        print(path, end="")

exec_bash_print("ls")

GenModels.ipynb
Untitled.ipynb
__pycache__
benckmark_data.ipynb
conf.py
logfile.log
untitled.txt
untitled1.txt


In [4]:
import logging
import logreset
from datetime import datetime


def setup():
    strtime = datetime.now().strftime("%Y%m%d_%H%M%S")
    logreset.reset_logging()
    filename=f'{conf.log_folder}{strtime}.log'
    logging.basicConfig(filename=filename, 
                        format='[%(asctime)s][%(levelname)s][python] %(message)s', 
                        datefmt='%Y-%m-%d_%H:%M:%S',  level=logging.DEBUG)
    
    logging.info(f"Logging info to {filename}")
    
    return filename, strtime

logfile, strtime = setup()


In [5]:
!cat $logfile

[2021-02-18_15:57:33][INFO][python] Logging info to /root/dev/2021-SIPTA-crema/logs/20210218_155733.log


In [5]:
def run_crema(filename:str,  target:int = 0, observed:str ="1", method:str = "cve",
              measure_time:bool = True, runs:int=1, warmups:int=0, timeout:int = 600, logfile:str=None,
              **kwargs):
        
    observed=str(observed)
    javafile=f"{conf.java_src}RunCrema.java"
    cmd = "-t" if measure_time else ""
    cmd += f" --method={method} -r {int(runs)} -w {int(warmups)} -x {int(target)}"
    if len(observed)>0: cmd+=f" -y {observed}"
    cmd += f" --timeout={timeout} {filename}"
    cmd += f" --log={logfile}"if logfile is not None else ""
    cmd= f"java -cp {conf.jar_file} {javafile} {cmd}"
    #print(cmd)

    output = exec_bash(cmd)
    
    exec(output[-1])
    return locals()["results"]


In [6]:
def relative_path(path):
    return path.replace(str(conf.prj_path), "./").replace("//","/")

relative_path(conf.model_folder)

'./networks/'

In [7]:
def get_model_path(vertex, kind, nodes, max_degree, max_indegree, max_values, num_vert, idx, **kwargs):
    p  = f""
    p += f"{relative_path(conf.model_folder)}vmodel/vmodel" if vertex else "hmodel/hmodel"
    p += f"-{kind}_n{nodes}_mID{max_indegree}_mD{max_degree}_mV{max_values}_nV{num_vert}-{idx}.uai"
    return p

def read_query(filename, query_type, field, **kwargs):
    p = filename.replace("vmodel/vmodel", query_type)\
                .replace("vmodel/vmodel", query_type)\
                .replace(relative_path(conf.model_folder), conf.query_folder)\
                .replace(".uai",".csv")
    out = pd.read_csv(p)[field].values[0]
    if np.isnan(out):
        return ""
    return out




params_dict = dict(
    idx = [1,2,3],
    num_vert = [2,4,6],
    max_degree = [6],
    max_indegree = [2,4,6],
    max_values = [4],
    nodes = [4, 5, 6, 7, 8, 9, 10],
    method=["cve"],
    kind = ["sing"],
    query_type = ["marg", "cond"]
)

func_params = {}
transformations = []


import expandas as expd


func_params = dict(
    filename = lambda t : get_model_path(**{**t, **{"vertex":t["method"].startswith("cve")}}),
    target = lambda t : read_query(**{**t, **dict(field="target")}),
    observed = lambda t : read_query(**{**t, **dict(field="observed")}),
    barren = lambda t : t["nodes"] - read_query(**{**t, **dict(field="size")}),
)
transformations = [lambda df: df.drop(columns=["idx"])]
params = expd.generate_input(params_dict, func_params, transformations)[:4]

params

,num_vert,max_degree,max_indegree,max_values,nodes,method,kind,query_type,filename,target,observed,barren
0,2,6,2,4,4,cve,sing,marg,./networks/vmodel/vmodel-sing_n4_mID2_mD6_mV4_...,3,,0
1,2,6,2,4,4,cve,sing,cond,./networks/vmodel/vmodel-sing_n4_mID2_mD6_mV4_...,0,3,0
2,2,6,2,4,5,cve,sing,marg,./networks/vmodel/vmodel-sing_n5_mID2_mD6_mV4_...,4,,0
3,2,6,2,4,5,cve,sing,cond,./networks/vmodel/vmodel-sing_n5_mID2_mD6_mV4_...,0,4,0


In [22]:
# single_experiemnt
t = params.iloc[0]
import expandas as expd


def run(start=0, end=None, **params_dict):

    global logfile, strtime, count
    
    logfile, strtime = setup()
    print(logfile)
    
    func_params = dict(
        filename = lambda t : get_model_path(**{**t, **{"vertex":t["method"].startswith("cve")}}),
        target = lambda t : read_query(**{**t, **dict(field="target")}),
        observed = lambda t : read_query(**{**t, **dict(field="observed")}),
        barren = lambda t : t["nodes"] - read_query(**{**t, **dict(field="size")}),
    )
    transformations = [lambda df: df.drop(columns=["idx"])]
    params = expd.generate_input(params_dict, func_params, transformations)
    
    end = end or len(params)
    params = params[start:end]

    count = 0

    def single_experiment(**t):
        global count, logfile

        logging.info((f"Starting experiment {count} out of {len(params)-1}"))
        count += 1
        
        output = t.copy()
        args = t.copy()
        args["filename"] = args["filename"].replace("./", str(conf.prj_path))
        
        # run query
        res = run_crema(**args, logfile=logfile)
        output["interval_result"] = " ".join([f"{p}" for p in res["interval_result"]]) if len(res["err_msg"])==0 else ""
        
        # save results for this experiment
        csvfile = f"{conf.res_folder}tmp/{strtime}.csv"
        logging.info(f"Results saved to {csvfile}")
        df = pd.DataFrame([output])
        if os.path.exists(csvfile):
            df.to_csv(csvfile, mode='a', header=False, index=False)
        else:
            df.to_csv(csvfile, index=False)
              
        return output


    df = pd.DataFrame(expd.Experiment(functions=[single_experiment]).run(params))
    
    return df


params_dict = dict(
    idx = [1,2,3],
    num_vert = [2,4,6],
    max_degree = [6],
    max_indegree = [2,4,6],
    max_values = [4],
    nodes = [4, 5, 6, 7, 8, 9, 10],
    method=["cve"],
    kind = ["sing"],
    query_type = ["marg", "cond"]
)

run(start=0, end=10, **params_dict)

# process output

/root/dev/2021-SIPTA-crema/logs/20210218_163402.log


,num_vert,max_degree,max_indegree,max_values,nodes,method,kind,query_type,filename,target,observed,barren,interval_result
0,2,6,2,4,4,cve,sing,marg,./networks/vmodel/vmodel-sing_n4_mID2_mD6_mV4_...,3,,0,0.476609993 0.614758876376 0.385241123624 0.52...
1,2,6,2,4,4,cve,sing,cond,./networks/vmodel/vmodel-sing_n4_mID2_mD6_mV4_...,0,3,0,0.21158887521400385 0.31777107907580027 0.4676...
2,2,6,2,4,5,cve,sing,marg,./networks/vmodel/vmodel-sing_n5_mID2_mD6_mV4_...,4,,0,0.139525240230232 0.40904841081014 0.233851725...
3,2,6,2,4,5,cve,sing,cond,./networks/vmodel/vmodel-sing_n5_mID2_mD6_mV4_...,0,4,0,0.08881519367445478 0.4106994478634178 0.20732...
4,2,6,2,4,6,cve,sing,marg,./networks/vmodel/vmodel-sing_n6_mID2_mD6_mV4_...,5,,0,0.06913445655785863 0.6015031986195333 0.09165...
5,2,6,2,4,6,cve,sing,cond,./networks/vmodel/vmodel-sing_n6_mID2_mD6_mV4_...,0,5,0,0.09281719321700743 0.8586191822095642 0.09701...
6,2,6,2,4,7,cve,sing,marg,./networks/vmodel/vmodel-sing_n7_mID2_mD6_mV4_...,6,,0,0.32631357354504925 0.8655622137396267 0.13443...
7,2,6,2,4,7,cve,sing,cond,./networks/vmodel/vmodel-sing_n7_mID2_mD6_mV4_...,0,6,0,0.06488227581822582 0.7364474616501411 0.26355...
8,2,6,2,4,8,cve,sing,marg,./networks/vmodel/vmodel-sing_n8_mID2_mD6_mV4_...,7,,0,0.017249948513106902 0.46850626627645087 0.062...
9,2,6,2,4,8,cve,sing,cond,./networks/vmodel/vmodel-sing_n8_mID2_mD6_mV4_...,0,7,0,0.16897875901639126 0.5845273842390771 0.09405...


In [ ]:
args.to_dict()

In [ ]:
n = 72
m = 5
x = 40


#print("1 - [" + " * ".join([f"(1 - ({x}/{n-i}))" for i in range(0,m)])+"]")
def get_prob(x):
    return 1 - np.prod([1 - x/(n-i) for i in range(0,m)])

In [14]:
output = {'num_vert': 2, 'max_degree': 6, 'max_indegree': 2, 'max_values': 4, 'nodes': 4, 'method': 'cve', 'kind': 'sing', 'query_type': 'marg', 'filename': './networks/vmodel/vmodel-sing_n4_mID2_mD6_mV4_nV2-1.uai', 'target': 3, 'observed': '', 'barren': 0, 'interval_result': '0.476609993 0.614758876376 0.385241123624 0.523390007'}

In [16]:
pd.DataFrame([output])

,num_vert,max_degree,max_indegree,max_values,nodes,method,kind,query_type,filename,target,observed,barren,interval_result
0,2,6,2,4,4,cve,sing,marg,./networks/vmodel/vmodel-sing_n4_mID2_mD6_mV4_...,3,,0,0.476609993 0.614758876376 0.385241123624 0.52...
